In [36]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import os

# Load the face detection classifier
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load the dataset
mask_dir = r'C:\Users\divsh\Downloads\archive (2)\images\masks'
no_mask_dir = r'C:\Users\divsh\Downloads\archive (2)\images\without_masks'

mask_images = [os.path.join(mask_dir, f) for f in os.listdir(mask_dir)]
no_mask_images = [os.path.join(no_mask_dir, f) for f in os.listdir(no_mask_dir)]

# Check that both mask and no-mask images are present in the dataset
if len(mask_images) == 0 or len(no_mask_images) == 0:
    raise ValueError("Both mask and no-mask images must be present in the dataset")

# Create labels for the dataset
mask_labels = np.ones(len(mask_images))
no_mask_labels = np.zeros(len(no_mask_images))

# Combine the datasets and labels
images = mask_images + no_mask_images
labels = np.concatenate([mask_labels, no_mask_labels])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.25, random_state=42)

# Define a function to extract features from the images
def extract_features(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    if len(faces) == 0:
        return None
    (x, y, w, h) = faces[0]
    face_roi = gray[y:y+h, x:x+w]
    face_roi = cv2.resize(face_roi, (100, 100))
    return face_roi.flatten()

# Extract features from the training set
X_train_features = []
y_train_subset = []
for i, image_path in enumerate(X_train):
    features = extract_features(image_path)
    if features is not None:
        X_train_features.append(features)
        y_train_subset.append(y_train[i])
        
X_train_features = np.array(X_train_features)
y_train_subset = np.array(y_train_subset)

# Check the number of unique classes in y_train
classes = np.unique(y_train_subset)
if len(classes) < 2:
    raise ValueError("At least two classes are required in the training data")

# Select a random subset of features that corresponds to the indices in y_train_subset
idx = np.random.choice(len(X_train_features), size=len(y_train_subset), replace=False)
X_train_subset = X_train_features[idx]

# Train the SVM model
svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train_subset, y_train_subset)

# Extract features from the testing set
X_test_features = []
y_test_subset = []
for i, image_path in enumerate(X_test):
    features = extract_features(image_path)
    if features is not None:
        X_test_features.append(features)
        y_test_subset.append(y_test[i])

X_test_features = np.array(X_test_features)
y_test_subset = np.array(y_test_subset)

# Test the SVM model
accuracy = svm.score(X_test_features, y_test_subset)
print('Accuracy:', accuracy)



Accuracy: 0.5324675324675324


In [37]:
# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (100, 100))
        features = face_roi.flatten().reshape(1, -1)

        # Predict the mask status using the SVM model
        mask_status = svm.predict(features)

        # Draw a rectangle around the face
        color = (0, 255, 0) if mask_status == 1 else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

        # Add a label to the rectangle
        label = "Wearing mask" if mask_status == 1 else "Not wearing mask"
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the resulting frame
    cv2.imshow('Face Mask Detection', frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close all windows
cap.release()
cv2.destroyAllWindows()
